In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#隠れ層のノード数
num_units = 1024

x = tf.placeholder(tf.float32, [None, 784])

#hidden1の計算、入力数のデータ数が2個から784に変わっただけ
w1 = tf.Variable(tf.truncated_normal([784, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, 10]))
b0 = tf.Variable(tf.zeros([10]))
#隠れ層の出力から、ソフトマックス関数を用いて確率を計算する。
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [4]:
t = tf.placeholder(tf.float32, [None, 10])
#誤差関数
loss = -tf.reduce_sum(t * tf.log(p))
#トレーニングアルゴリズム
train_step = tf.train.AdamOptimizer().minimize(loss)
#正解率を出すために計算？
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
#正解率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
#セッションの用意
sess = tf.InteractiveSession()
#Variableの初期化
sess.run(tf.initialize_all_variables())

In [6]:
i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

Step: 100, Loss: 3136.286133, Accuracy: 0.906700
Step: 200, Loss: 2440.697021, Accuracy: 0.928000
Step: 300, Loss: 1919.005005, Accuracy: 0.941900
Step: 400, Loss: 1982.860596, Accuracy: 0.939400
Step: 500, Loss: 1734.469604, Accuracy: 0.945500
Step: 600, Loss: 1377.526245, Accuracy: 0.956100
Step: 700, Loss: 1332.826050, Accuracy: 0.960600
Step: 800, Loss: 1184.056641, Accuracy: 0.963600
Step: 900, Loss: 1134.456177, Accuracy: 0.964700
Step: 1000, Loss: 1236.610229, Accuracy: 0.961900
Step: 1100, Loss: 1116.115234, Accuracy: 0.965400
Step: 1200, Loss: 1125.281006, Accuracy: 0.964600
Step: 1300, Loss: 1194.283691, Accuracy: 0.961800
Step: 1400, Loss: 1100.531494, Accuracy: 0.966700
Step: 1500, Loss: 1061.845215, Accuracy: 0.969200
Step: 1600, Loss: 1111.877563, Accuracy: 0.966600
Step: 1700, Loss: 953.381348, Accuracy: 0.972000
Step: 1800, Loss: 958.325256, Accuracy: 0.971000
Step: 1900, Loss: 1033.686279, Accuracy: 0.968000
Step: 2000, Loss: 991.070801, Accuracy: 0.970900


In [ ]:
images, labels = mnist.test.images, mnist.test.labels
p_val = sess.run(p, feed_dict={x:images, t: labels}) 

fig = plt.figure(figsize=(8,15))
for i in range(10):
    c = 1
    for (image, label, pred) in zip(images, labels, p_val):
        prediction, actual = np.argmax(pred), np.argmax(label)
        if prediction != i:
            continue
        if (c < 4 and i == actual) or (c >= 4 and i != actual):
            subplot = fig.add_subplot(10,6,i*6+c)
            subplot.set_xticks([])
            subplot.set_yticks([])
            subplot.set_title('%d / %d' % (prediction, actual))
            subplot.imshow(image.reshape((28,28)), vmin=0, vmax=1,
                           cmap=plt.cm.gray_r, interpolation="nearest")
            c += 1
            if c > 6:
                break
plt.show()